# Tarea PySpark

### Objetivo:
 Analizar la eficiencia de los jugadores en términos generales y por posición, así como determinar la contribución al equipo por jugador tomando en cuenta los datos obtenidos

Usarás la base de datos del archivo 'fusbol.csv' para obtener tus datos. Checa la estructura del archivo para ver si es necesario limpiar la informacion, ver su estructura y así sea más fácil completar la tarea. Besos.

### Ejercicio 1:
Carga la base de datos en un DataFrame de Pyspark (con 2 nucleos). Valida los rangos de los valores donde sea aplicable, así como su corrección (en caso de ser necesaria). 
Después, utilizando las variables más relevantes como 'Ast/90', 'PassCmp%', etc., concluye qué ligas tienen los mejores jugadores por posición. Es decir, si los mejores jugadores defensas son de la liga francesa, inglesa, etc., por ejemplo.
Como cada persona tiene una definición de "mejor", utiliza las siguientes metricas por posición:
- Delanteros: npG+A/90 y npxG+xA/90
- Medios: KeyPass/90 y PassCmp%
- Defensas: PressSucc% y Interceptions/90

Con los resultados obtenidos, grafica por posición para que tu conclusión tenga un respaldo visual también.

In [ ]:
import findspar
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('firstSession')\
    .config('spark.master', 'local[2]')\
    .config('spark.executor.memory', '1g')\
    .config("spark.sql.shuffle.partitions", 10)\
    .config('spark.driver.memory','1g')\
    .getOrCreate()
spark

In [ ]:
df_c = spark.read.csv('fusbol.csv', header=True, inferSchema=True)
df_c.show()

In [ ]:
df_c.printSchema()

In [ ]:
from pyspark.sql.functions import avg, desc
delanteros_por_liga = df_c.filter(df_c["Pos"] == "FW").groupby("Comp").agg(
    avg("npG+A/90").alias("Promedio_npG_A_90"),
    avg("npxG+xA/90").alias("Promedio_npxG_xA_90")
).orderBy(desc("Promedio_npG_A_90"), desc("Promedio_npxG_xA_90"))

delanteros_por_liga.show()

medios_por_comp = df_c.filter(df_c["Pos"] == "MF").groupby("Comp").agg(
    avg(df_c["KeyPass/90"]).alias("Promedio_KeyPass_90"),
    avg(df_c["PassCmp%"]).alias("Promedio_PassCmp_porcentaje")
).orderBy(desc("Promedio_KeyPass_90"), desc("Promedio_PassCmp_porcentaje"))
medios_por_comp.show()


defensas_por_comp = df_c.filter(df_c["Pos"] == "DF").groupby("Comp").agg(
    avg(df_c["PressSucc%"]).alias("Promedio_PressSucc_porcentaje"),
    avg(df_c["Interceptions/90"]).alias("Promedio_Interceptions_90")
).orderBy(desc("Promedio_PressSucc_porcentaje"), desc("Promedio_Interceptions_90"))
defensas_por_comp.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
df_delanteros_pandas= delanteros_por_comp.toPandas()
df_medios_pandas= medios_por_comp.toPandas()
df_defensas_pandas= defensas_por_comp.toPandas()

print(df_delanteros_pandas)


# Gráfico para delanteros
fig, ax = plt.subplots(figsize=(10, 6))
bar_width = 0.35
index = np.arange(len(df_delanteros_pandas["Comp"]))

plt.barh(index, df_delanteros_pandas["Promedio_npG_A_90"], bar_width, color='skyblue', label='Promedio npG+A/90')
plt.barh(index + bar_width, df_delanteros_pandas["Promedio_npxG_xA_90"], bar_width, color='orange', label='Promedio npxG+xA/90')

plt.xlabel('Promedio')
plt.ylabel('Competición')
plt.title('Comparación de métricas para delanteros por competición')
plt.yticks(index + bar_width / 2, df_delanteros_pandas["Comp"])
plt.legend()

plt.show()

# Gráfico para medios
fig, ax = plt.subplots(figsize=(10, 6))
bar_width = 0.35
index = np.arange(len(df_medios_pandas["Comp"]))

plt.barh(index, df_medios_pandas["Promedio_KeyPass_90"], bar_width, color='skyblue', label='Promedio KeyPass/90')
plt.barh(index + bar_width, df_medios_pandas["Promedio_PassCmp_porcentaje"], bar_width, color='orange', label='Promedio PassCmp%')

plt.xlabel('Promedio')
plt.ylabel('Competición')
plt.title('Comparación de métricas para medios por competición')
plt.yticks(index + bar_width / 2, df_medios_pandas["Comp"])
plt.legend()

plt.show()

# Gráfico para defensas
fig, ax = plt.subplots(figsize=(10, 6))
bar_width = 0.35
index = np.arange(len(df_defensas_pandas["Comp"]))

plt.barh(index, df_defensas_pandas["Promedio_PressSucc_porcentaje"], bar_width, color='skyblue', label='Promedio PressSucc%')
plt.barh(index + bar_width, df_defensas_pandas["Promedio_Interceptions_90"], bar_width, color='orange', label='Promedio Interceptions/90')

plt.xlabel('Promedio')
plt.ylabel('Competición')
plt.title('Comparación de métricas para defensas por competición')
plt.yticks(index + bar_width / 2, df_defensas_pandas["Comp"])
plt.legend()

plt.show()

### Ejercicio 2:

Ahora hagamos algo un poco más interesante. Escoge algún jugador de todos los disponibles y toma 5 metricas, las que quieras. Debes concluir en qué percentil se encuentra el jugador en esas métricas que escogiste. Obviamente, vas a comparar sus valores con todos los demás con los que comparte posición y liga, para no tener un sesgo y que la información no pierda robustez. Por último, genera un DataFrame de Pyspark con todos los datos solicitados. Muestra el DataFrame y conviertelo a otro de tipo pandas. Muestra los dos.

In [ ]:
import pandas as pd
import pyspark.sql.functions as F
from pyspark.sql import Window
import matplotlib.pyplot as plt

# Encontrar la fila de Mbappé en el DataFrame
mbappe_row = df_c.filter(F.col("Player") == "Kylian Mbappé").collect()[0]

# Extraer los valores de las métricas para Mbappé
mbappe_values = {
    'Shots/90': mbappe_row["Shots/90"],  # Cambiado de 'npG/90' a 'Shots/90'
    'npG+A/90': mbappe_row["npG+A/90"],
    'npG/90': mbappe_row["npG/90"],
    'SoTs/90': mbappe_row["SoTs/90"],
    'xA/90': mbappe_row["xA/90"]
}

# Filtrar los datos para incluir solo los delanteros de la Ligue 1
ligue1_forwards_df = df_c.filter((F.col("Comp") == "fr Ligue 1") & (F.col("Pos") == "FW"))

# Métricas a considerar
metrics = ["Shots/90", "npG+A/90", "npG/90", "SoTs/90", "xA/90"]  # Cambiado 'npG/90' por 'Shots/90'

# Crear una lista para almacenar los resultados
results = []

# Comparar los valores de Kylian Mbappé con estos percentiles
for metric in metrics:
    quantile_df = ligue1_forwards_df.select(metric, F.percent_rank().over(Window.orderBy(F.desc(metric))).alias(f"{metric}_quantile"))
    mbappe_quantile = quantile_df.filter(F.col(metric) == mbappe_values[metric]).select(f"{metric}_quantile").collect()[0][0]
    results.append({'Categoria': metric, 'Percentil': mbappe_quantile * 100})

#Df pyspark
results_rows = [Row(Categoria=result['Categoria'], Percentil=result['Percentil']) for result in results]
results_df_spark = spark.createDataFrame(results_rows)
results_df_spark.show()

#Df pandas
results_df = pd.DataFrame(results)
print(results_df)

#ver quien es el delanteero de la ligue1 que tiene mas npG/90 para justificar el 0% en ell percentil
fw_ligue1_df = df_c.filter((F.col("Pos") == "FW") & (F.col("Comp") == "fr Ligue 1"))
mejor_delantero = fw_ligue1_df.orderBy(F.desc("npG/90")).select("Player", "npG/90").first()
print("El mejor delantero de la fr Ligue 1 en términos de npG/90 es:")
print(mejor_delantero)
results_df = results_df.sort_values(by='Percentil', ascending=True)

#Graficar stats
plt.figure(figsize=(10,6))
plt.barh(results_df['Categoria'], results_df['Percentil'], color='skyblue')
plt.xlabel('Percentil')
plt.title('Percentiles de Kylian Mbappé en comparación con otros delanteros de la Ligue 1')
plt.grid(axis='x')
plt.show()